In [52]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision.transforms import ToTensor

import matplotlib.pyplot as plt
import numpy as np

# use folder as train data
from torchvision.datasets import ImageFolder
from torchvision.transforms import ToTensor

train_data = ImageFolder(root='images/impact_dataset', transform=ToTensor())
train_data_loader = DataLoader(train_data, batch_size=1, shuffle=True)





In [53]:
for X, y in train_data_loader:
    print('Shape of X:', X.shape)
    print('Shape of y:', y.shape, y.dtype)
    break
    

Shape of X: torch.Size([1, 3, 500, 500])
Shape of y: torch.Size([1]) torch.int64


In [54]:
# Utilisation de la GPU si disponible
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)


cpu


In [55]:

# Définition du modèle
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten() # convert 2D image to 1D vector
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(3 * 500 * 500, 512),  # Corrected input size
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 50),
            nn.ReLU()
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=750000, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=50, bias=True)
    (5): ReLU()
  )
)


In [56]:
# Définition de la fonction de coût et de l'optimiseur
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)


In [57]:

# Définition de la fonction d'entraînement
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        
        # Calcul de la prédiction et de la fonction de coût
        X, y = X.to(device), y.to(device)
        pred = model(X)
        loss = loss_fn(pred, y)
        
        # Rétropropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


In [58]:

# Définition de la fonction de test
def test(dataloader, model):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    
    test_loss /= num_batches
    correct /= size
    
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")


In [59]:
epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_data_loader, model, loss_fn, optimizer)
    test(train_data_loader, model)
print("Done!")


Epoch 1
-------------------------------
loss: 3.934689  [    0/80000]
loss: 3.562480  [  100/80000]


KeyboardInterrupt: 

In [ ]:
torch.save(model.state_dict(), "model.pth")
print("Saved PyTorch Model State to model.pth")


In [ ]:
model = NeuralNetwork()
model.load_state_dict(torch.load("model.pth"))

